In [1]:
! pip install tensorflow


^C


Defaulting to user installation because normal site-packages is not writeable
  Using cached h5py-3.12.1-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/390.3 MB 1.5 MB/s eta 0:04:16
   ---------------------------------------- 0.8/390.3 MB 1.5 MB/s eta 0:04:28
   ---------------------------------------- 1.3/390.3 MB 1.5 MB/s eta 0:04:27
   ---------------------------------------- 1.6/390.3 MB 1.5 MB/s eta 0:04:25
   ---------------------------------------- 1.8/390.3 MB 1.4 MB/s eta 0:04:35
   ---------------------------------------- 2.1/390.3 MB 1.4 MB/s eta 0:04:38
   ---------------------------------------- 2.4/390.3 MB 1.4 MB/s eta 0:04:44
   ---------------------------------------- 2.6/390.3 MB 1.4 MB/s eta 0:04:35
   -----------

In [75]:
import numpy as np
import cv2

In [76]:
from tensorflow.keras.models import load_model

# Filepath to the saved model
model_path = 'covid_detection_model.h5'

# Load the model
model = load_model(model_path)

# Check the model summary to confirm it was loaded correctly
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 98, 98, 300)    │         8,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 96, 96, 300)    │       810,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 94, 94, 250)    │       675,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 250)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 13, 13, 200)    │       450,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 11, 11, 100)    │       180,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 9, 9, 80)       │        72,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 80)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        20,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,266,992 (8.65 MB)

 Trainable params: 2,266,990 (8.65 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [77]:
def convert_model_output(model_output, num ):
    """
    Convert model numeric output to descriptive labels with an 'extra' field.
    
    :param model_output: Numeric output from the model (0, 1, 2, 3).
    :return: Dictionary with descriptive label and an 'extra' field set to True.
    """
    case_codes = {0: 'Normal', 1: 'COVID', 2: 'Lung_Opacity', 3: 'Viral Pneumonia'}
    
    # Retrieve the corresponding label
    label = case_codes.get(model_output, 'Unknown')
    
    # Return the label with an 'extra' field set to True
    if num == 0: 
        return { 'chest_xray'  : label,
            "only": True,}
    return { 'chest_xray'  : label,
            }

# Example usage
model_output = 0  # Assume the model predicts 'Lung_Opacity'
converted_output = convert_model_output(model_output,0)
print(converted_output)

{'chest_xray': 'Normal', 'only': True}


In [78]:
def ask_for_input():
    print("Do you have an image, no image, or both?")
    choice = input("Enter 'image', 'no image', or 'both': ").strip().lower()
    return choice

In [79]:
def ask_symptoms(symptoms):
    print("Please add your symptoms:")
    num = input("Enter number of symptoms: ")
    for i in range(int(num)):
        symptom = input(f"Enter symptom {i+1} (comma separated): ").strip().lower()
        symptoms[symptom] = True  # Add symptom to the dictionary
    return symptoms



In [80]:

    # Rule-based expert system
rules_with_explanation = [
    {'conditions': {'fever': True, "cough": True}, 'conclusion': 'flu', 'explanation': {"fever and cough are common flu symptoms."}},
    {'conditions': {'nausea': True, "vomiting": True}, 'conclusion': 'food poisoning', 'explanation': {"Nausea and vomiting are typical for food poisoning."}},
    {'conditions': {'fever': True, "shortness_of_breath": True, "fatigue": True}, 'conclusion': 'COVID', 'explanation': {"These symptoms are indicative of COVID-19."}},
    {'conditions': {'chest_xray': 'Normal', 'fever': True, "cough": True, "shortness_of_breath": True}, 'conclusion': 'Lung_Opacity', 'explanation': {"Chest X-ray shows lung opacity, indicating possible pneumonia."}},
    {'conditions': {'chest_xray': 'Normal', 'fever': True, "cough": True, "fatigue": True}, 'conclusion': 'Viral Pneumonia', 'explanation': {"Chest X-ray shows signs of viral pneumonia."}},
    {'conditions': {'fever': True, "headache": True, "muscle_aches": True}, 'conclusion': 'viral infection', 'explanation': {"Fever, headache, and muscle aches are indicative of a viral infection."}},
    {'conditions': {'fever': True, "abdominal_pain": True, "diarrhea": True}, 'conclusion': 'gastroenteritis', 'explanation': {"Abdominal pain, fever, and diarrhea suggest gastroenteritis."}},
    {'conditions': {'fever': True, "jaundice": True}, 'conclusion': 'hepatitis', 'explanation': {"Fever and jaundice indicate possible hepatitis."}},
    {'conditions': {'fever': True, "sore_throat": True, "swollen_lymph_nodes": True}, 'conclusion': 'streptococcal throat infection', 'explanation': {"These symptoms are indicative of a streptococcal throat infection."}},
    {'conditions': {'fever': True, "chills": True, "body_aches": True}, 'conclusion': 'influenza', 'explanation': {"Fever, chills, and body aches are common symptoms of influenza."}},
    {'conditions': {'fever': True, "rash": True, "joint_pain": True}, 'conclusion': 'chikungunya', 'explanation': {"Fever, rash, and joint pain suggest chikungunya."}},
    {'conditions': {'fever': True, "confusion": True, "altered_mental_status": True}, 'conclusion': 'meningitis', 'explanation': {"These symptoms are indicative of meningitis."}},
    {'conditions': {'fever': True, "back_pain": True, "urinary_urgency": True}, 'conclusion': 'urinary tract infection (UTI)', 'explanation': {"Fever, back pain, and urinary urgency suggest a urinary tract infection."}},
    {'conditions': {'fever': True, "exposure_to_infected_person": True}, 'conclusion': 'exposure to contagious disease', 'explanation': {"Fever and known exposure to an infected person suggest a contagious disease."}},
    {'conditions': {'fever': True, "recent_travel": True, "cough": True}, 'conclusion': 'travel-related illness', 'explanation': {"Fever, cough, and recent travel indicate a travel-related illness."}},
    {'conditions': {'fever': True, "chest_pain": True, "cough": True}, 'conclusion': 'pulmonary embolism', 'explanation': {"These symptoms may indicate a pulmonary embolism."}},
    {'conditions': {'fever': True, "pain_urination": True}, 'conclusion': 'urethritis', 'explanation': {"Fever and painful urination suggest urethritis."}},
    {'conditions': {'fever': True, "night_sweats": True, "cough": True}, 'conclusion': 'tuberculosis (TB)', 'explanation': {"Fever, night sweats, and cough are indicative of tuberculosis."}},
    {'conditions': {'fever': True, "exposure_to_animals": True, "rash": True}, 'conclusion': 'zoonotic disease', 'explanation': {"Fever and exposure to animals suggest a zoonotic disease."}},
    {'conditions': {'fever': True, "sudden_shortness_of_breath": True, "chest_pain": True}, 'conclusion': 'heart attack', 'explanation': {"These symptoms may indicate a heart attack."}},
    {'conditions': {'chest_xray': 'Normal',"only": True,}, 'conclusion': 'normal', 'explanation': {"Chest X-ray appears normal, no visible abnormalities."}},
    {'conditions': {'chest_xray': 'Lung_Opacity',"only": True,}, 'conclusion': 'pneumonia', 'explanation': {"Chest X-ray shows lung opacity, indicating possible pneumonia."}},
    {'conditions': {'chest_xray': 'Viral Pneumonia',"only": True,}, 'conclusion': 'viral pneumonia', 'explanation': {"Chest X-ray shows signs of viral pneumonia."}},
    {'conditions': {'chest_xray': 'COVID',"only": True,}, 'conclusion': 'COVID', 'explanation': {"Chest X-ray shows features consistent with COVID-19."}},
    { 'conditions': {'chest_xray': 'Normal', 'fever': False, "cough": False, "shortness_of_breath": False}, 'conclusion': 'Normal', 'explanation': {"Chest X-ray is clear. No signs of abnormality detected."} },
    { 'conditions': {'chest_xray': 'Normal', 'fever': True, "cough": True, "fatigue": True}, 'conclusion': 'Viral Pneumonia', 'explanation': {"Chest X-ray shows signs of viral pneumonia."} },
    { 'conditions': {'chest_xray': 'Abnormal', 'fever': True, "cough": True, "shortness_of_breath": True, "loss_of_taste_or_smell": True}, 'conclusion': 'COVID-19', 'explanation': {"Chest X-ray shows ground-glass opacities, which are indicative of COVID-19 infection."} }, 
    { 'conditions': {'chest_xray': 'Normal', 'fever': True, "cough": True, "shortness_of_breath": True}, 'conclusion': 'Lung Opacity', 'explanation': {"Chest X-ray shows lung opacity, indicating possible pneumonia."} }
]

In [81]:
def forward_chaining_with_explanation(symptoms):
    for rule in rules_with_explanation:
        if all(symptoms.get(cond, False) == value for cond, value in rule['conditions'].items() ):
            return rule["conclusion"], rule["explanation"]
    return "No diagnosis found .", "No sufficient evidence."

In [82]:
def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image = cv2.resize(image, target_size) 
    image = image.astype('float32') / 255.0 # rescale 
    image = np.expand_dims(image, axis=0) # add batch dimension return image
    return image    

In [84]:
def main():
    choice = ask_for_input()
    symptoms = {}
    if choice == '0':
        print("Please upload your image path.")
        path= input('add file path')
        target_size = (100, 100)
        image = preprocess_image(path, target_size)
        predic = model.predict(image)
        prediction = np.argmax(predic, axis=1)
        print(prediction)
        symptoms = convert_model_output(int(prediction),0)
        print(symptoms)
        diagnosis, explanation =  forward_chaining_with_explanation(symptoms)
        print(f"Diagnosis: {diagnosis}")
        print(f"Diagnosis: {explanation}") 
    elif choice == '1':
        print("Please add your symptoms.")
        
            
        symptoms = ask_symptoms(symptoms)
        print(symptoms)

        diagnosis, explanation =  forward_chaining_with_explanation(symptoms)
        

        print(f"Diagnosis: {diagnosis}")
        print(f"Diagnosis: {explanation}")
        # symptoms = ask_symptoms(symptoms)
        # prediction_format = predict_from_symptoms(symptoms)   
    else:
        print("add your image path symptoms.")
        path= input('add file path')
        target_size = (100, 100)
        image = preprocess_image(path, target_size)
        predic = model.predict(image)
        prediction = np.argmax(predic, axis=1)
        print(prediction)
        symptoms = convert_model_output(int(prediction),1)
        symptoms = ask_symptoms(symptoms)

        diagnosis, explanation =  forward_chaining_with_explanation(symptoms)
        print(f"Diagnosis: {diagnosis}")
        print(f"Diagnosis: {explanation}")
        # symptoms = ask_symptoms()
        # prediction_format = predict_from_model_and_symptoms(choice 
    
    

if __name__ == "__main__":
    main()

Do you have an image, no image, or both?
Please add your symptoms.
Please add your symptoms:
{'fever': True, 'vomiting': True, 'fatigue': True}
Diagnosis: No diagnosis found .
Diagnosis: No sufficient evidence.


In [ ]:
# import streamlit as st
# import numpy as np
# from keras.models import load_model

# # def preprocess_image(path, target_size):
# #     # Add your image preprocessing logic here
# #     pass

# # def convert_model_output(prediction, mode):
# #     # Convert model output into a readable format for symptoms
# #     pass

# # def ask_symptoms(symptoms):
# #     # Function to collect symptoms from the user via Streamlit widgets
# #     symptoms['fever'] = st.checkbox('Do you have a fever?', value=False)
# #     symptoms['cough'] = st.checkbox('Do you have a cough?', value=False)
# #     symptoms['shortness_of_breath'] = st.checkbox('Do you have shortness of breath?', value=False)
# #     symptoms['fatigue'] = st.checkbox('Do you feel fatigued?', value=False)
# #     return symptoms

# # def forward_chaining_with_explanation(symptoms):
# #     # Add your diagnosis logic here
# #     return "Sample Diagnosis", "Sample Explanation"

# # # Load your trained model
# # model = load_model('path_to_your_model.h5')

# def main():
#     st.title("Medical Diagnosis App")
#     st.write("This app helps you diagnose conditions based on symptoms and/or medical images.")

#     choice = st.radio("Choose an option:", ("Upload Image", "Enter Symptoms", "Both"))
#     symptoms = {}

#     if choice == "Upload Image":
#         st.write("Please upload your medical image.")
#         uploaded_file = st.file_uploader("Choose a file", type=["jpg", "jpeg", "png"])
#         if uploaded_file is not None:
#             target_size = (100, 100)
#             image = preprocess_image(uploaded_file, target_size)
#             predic = model.predict(image)
#             prediction = np.argmax(predic, axis=1)
#             st.write(f"Model Prediction: {prediction}")
#             symptoms = convert_model_output(int(prediction), 0)
#             st.write("Extracted Symptoms:", symptoms)

#             diagnosis, explanation = forward_chaining_with_explanation(symptoms)
#             st.subheader("Diagnosis")
#             st.write(diagnosis)
#             st.subheader("Explanation")
#             st.write(explanation)

#     elif choice == "Enter Symptoms":
#         st.write("Please provide your symptoms.")
#         symptoms = ask_symptoms(symptoms)
#         st.write("Your Symptoms:", symptoms)

#         diagnosis, explanation = forward_chaining_with_explanation(symptoms)
#         st.subheader("Diagnosis")
#         st.write(diagnosis)
#         st.subheader("Explanation")
#         st.write(explanation)

#     elif choice == "Both":
#         st.write("Please upload your medical image and provide symptoms.")
#         uploaded_file = st.file_uploader("Choose a file", type=["jpg", "jpeg", "png"])
#         if uploaded_file is not None:
#             target_size = (100, 100)
#             image = preprocess_image(uploaded_file, target_size)
#             predic = model.predict(image)
#             prediction = np.argmax(predic, axis=1)
#             st.write(f"Model Prediction: {prediction}")
#             symptoms = convert_model_output(int(prediction), 1)

#         symptoms = ask_symptoms(symptoms)
#         st.write("Combined Symptoms:", symptoms)

#         diagnosis, explanation = forward_chaining_with_explanation(symptoms)
#         st.subheader("Diagnosis")
#         st.write(diagnosis)
#         st.subheader("Explanation")
#         st.write(explanation)

# if __name__ == "__main__":
#     main()


2024-12-19 10:19:39.051 
  command:

    streamlit run C:\Users\dodom\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-19 10:19:39.054 Session state does not function when running a script without `streamlit run`


In [73]:
# import streamlit as st

# # Set page configuration
# st.set_page_config(
#     page_title="Streamlit UI Example",
#     page_icon="🌐",
#     layout="wide",
# )

# # Sidebar for navigation
# st.sidebar.title("Navigation")
# pages = ["Home", "Upload File", "Analysis", "About"]
# selection = st.sidebar.radio("Go to", pages)
# uploaded_file=''

# # Home Page
# if selection == "Home":
#     st.title("Welcome to the Streamlit UI Example! 🌟")
#     st.write("""
#         This is a demo of a Streamlit application. Use the navigation menu on the left to explore different sections:
#         - Upload a file for analysis
#         - View analysis results
#         - Learn more about the project
#     """)


# # Upload File Page
# elif selection == "Upload File":
#     st.title("Upload Your File 📂")
#     uploaded_file = st.file_uploader("Choose a file", type=["csv", "xlsx", "txt"])
#     if uploaded_file is not None:
#         st.success(f"File {uploaded_file.name} uploaded successfully!")

# # Analysis Page
# elif selection == "Analysis":
#     st.title("Data Analysis 🔍")
#     st.write("This section will display analysis results.")
#     if uploaded_file is not None:
#         st.write("Performing analysis on the uploaded file...")
#         # Placeholder for analysis logic
#     else:
#         st.warning("Please upload a file first.")

# # About Page
# elif selection == "About":
#     st.title("About This App 📖")
#     st.write("""
#         This app is built with Streamlit, a Python framework for creating interactive web applications.
#         Streamlit makes it easy to build and share data-driven tools.
#     """)

# # Footer
# st.sidebar.markdown("---")
# st.sidebar.write("Built with ❤️ using [Streamlit](https://streamlit.io/).")


In [ ]:
# import streamlit as st

# # Title of the app
# st.title('Simple Streamlit App')

# # Text input
# user_input = st.text_input('Enter your name:', 'Type here')

# # Display the user input
# st.write(f'Hello, {user_input}!')